In [5]:
!pip install fake-useragent aiohttp requests beautifulsoup4 pandas tqdm

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import os
import re
import concurrent.futures
from tqdm import tqdm

# 사용자 에이전트 목록 (차단 방지용)
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.2 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 16_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.2 Mobile/15E148 Safari/604.1'
]

# CSV 파일명 설정
BASE_FILENAME = 'dcinside_chartanalysis_data'

# 페이지 URL 포맷
BASE_URL = 'https://gall.dcinside.com/mgallery/board/lists/'
VIEW_URL = 'https://gall.dcinside.com/mgallery/board/view/'

# 총 크롤링할 게시물 수
TOTAL_POSTS = 20000
# 중간 저장 단위
SAVE_INTERVAL = 5000

# 게시물 ID 캐시 (중복 방지)
post_id_cache = set()

# 현재까지 수집한 게시물 수
collected_posts = 0

# 페이지 범위 설정 (566~25824)
start_page = 25115
end_page = 26030

# 병렬 처리 설정
MAX_WORKERS = 5  # 동시에 처리할 상세 페이지 요청 수
MAX_RETRIES = 3   # 요청 실패 시 재시도 횟수
MIN_DELAY = 0.1   # 최소 딜레이(초)
MAX_DELAY = 0.5   # 최대 딜레이(초)

# 세션 객체 생성 (연결 재사용)
session = requests.Session()

# 함수: 랜덤 딜레이 적용 (더 짧은 딜레이)
def random_delay():
    delay = random.uniform(MIN_DELAY, MAX_DELAY)
    time.sleep(delay)

# 함수: 요청 헤더 생성
def get_headers():
    return {
        'User-Agent': random.choice(user_agents),
        'Referer': 'https://gall.dcinside.com/mgallery/board/lists/?id=chartanalysis',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0'
    }

# 함수: CSV 저장
def save_to_csv(dataframe, count):
    filename = f"{BASE_FILENAME}_{count}.csv"
    dataframe.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"{filename} 파일에 {len(dataframe)}개 게시물 저장 완료")

# 함수: 게시물 상세 페이지 크롤링 (재시도 로직 포함)
def crawl_post_detail(post_info):
    post_url, post_id = post_info
    
    for attempt in range(MAX_RETRIES):
        try:
            # 너무 빠른 요청을 피하기 위한 최소 딜레이
            if attempt > 0:
                time.sleep(random.uniform(0.5, 1.0))
            
            headers = get_headers()
            response = session.get(post_url, headers=headers, timeout=5)
            
            if response.status_code != 200:
                continue
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 내용 추출 (최적화: 정확한 선택자 사용)
            content_div = soup.select_one('div.write_div')
            content = content_div.text.strip() if content_div else "내용 없음"
            
            # 조회수 추출
            view_count = soup.select_one('span.gall_count')
            view_count = view_count.text.replace('조회 ', '').strip() if view_count else "0"
            
            # 댓글 수 추출
            comment_count = soup.select_one('span.gall_reply_num')
            comment_count = comment_count.text.replace('댓글 ', '').strip() if comment_count else "0"
            
            # 숫자만 추출 (컴파일된 정규식 패턴 사용)
            view_count = re.sub(r'[^0-9]', '', view_count)
            comment_count = re.sub(r'[^0-9]', '', comment_count)
            
            return {
                'post_id': post_id,
                '내용': content,
                '조회수': view_count,
                '댓글갯수': comment_count
            }
            
        except Exception as e:
            if attempt == MAX_RETRIES - 1:
                # print(f"게시물 상세 페이지 크롤링 실패 (최대 재시도 횟수 초과): {post_url}")
                return None
    
    return None

# 게시글 번호 기준으로 균등하게 샘플링할 범위 계산
def calculate_sampling_interval():
    # 전체 페이지 범위의 예상 게시물 수
    total_pages = end_page - start_page + 1
    estimated_total_posts = total_pages * 100  # 페이지당 100개 게시물
    
    # 샘플링 간격 계산
    sampling_interval = max(1, estimated_total_posts // TOTAL_POSTS)
    return sampling_interval

# 함수: 게시판 목록 페이지에서 게시물 정보 추출
def extract_post_info(page):
    for retry in range(MAX_RETRIES):
        try:
            headers = get_headers()
            page_url = f"{BASE_URL}?id=chartanalysis&page={page}&list_num=100&sort_type=N"
            response = session.get(page_url, headers=headers, timeout=5)
            
            if response.status_code != 200:
                continue
            
            soup = BeautifulSoup(response.text, 'html.parser')
            posts = soup.select('tr.ub-content')
            
            post_info_list = []
            
            for post in posts:
                # 공지사항 건너뛰기
                if 'notice' in post.get('class', []):
                    continue
                
                # 게시물 정보 추출
                post_id_elem = post.select_one('td.gall_num')
                post_id = post_id_elem.text.strip() if post_id_elem else "0"
                
                # 이미 수집한 게시물인지 확인
                if post_id in post_id_cache:
                    continue
                
                post_title_elem = post.select_one('td.gall_tit a')
                title = post_title_elem.text.strip() if post_title_elem else "제목 없음"
                
                post_date_elem = post.select_one('td.gall_date')
                date = post_date_elem.text.strip() if post_date_elem else ""
                
                # 게시물 URL 생성
                post_detail_url = None
                if post_title_elem and post_title_elem.get('href'):
                    post_detail_url = post_title_elem['href']
                    if not post_detail_url.startswith('http'):
                        post_detail_url = "https://gall.dcinside.com" + post_detail_url
                
                if post_detail_url:
                    post_info_list.append({
                        'post_id': post_id,
                        'title': title,
                        'date': date,
                        'url': post_detail_url
                    })
            
            return post_info_list
            
        except Exception as e:
            if retry == MAX_RETRIES - 1:
                # print(f"페이지 {page} 크롤링 실패: {e}")
                return []
            time.sleep(random.uniform(1.0, 2.0))
    
    return []

# 메인 크롤링 함수
def main():
    global collected_posts, post_id_cache
    
    all_data = []  # 수집된 모든 게시물 데이터
    
    # 기존 파일 체크 및 이어서 크롤링
    existing_files = [f for f in os.listdir('.') if f.startswith(BASE_FILENAME) and f.endswith('.csv')]
    if existing_files:
        latest_file = sorted(existing_files)[-1]
        temp_df = pd.read_csv(latest_file, encoding='utf-8-sig')
        collected_posts = len(temp_df)
        all_data = temp_df.to_dict('records')  # 리스트로 변환하여 저장
        print(f"기존 파일 {latest_file}에서 {collected_posts}개 게시물 불러옴")
        
        # 이미 수집한 게시물 ID 캐시에 추가
        for post_id in temp_df['번호']:
            post_id_cache.add(str(post_id))
    
    # 게시물 샘플링 간격 계산
    sampling_interval = calculate_sampling_interval()
    print(f"샘플링 간격: 약 {sampling_interval}개 게시물마다 1개 수집")
    
    # 현재 수집한 게시물 수 확인
    current_batch_size = collected_posts % SAVE_INTERVAL
    
    # 각 페이지에서 게시물 목록 추출을 위한 페이지 범위 생성
    page_ranges = list(range(start_page, end_page + 1))
    
    # 크롤링 시작 시간 기록
    start_time = time.time()
    posts_at_start = collected_posts
    
    try:
        # 메인 크롤링 루프
        with tqdm(total=TOTAL_POSTS, initial=collected_posts, desc="게시물 크롤링 진행") as pbar:
            # 페이지 단위로 처리
            for page_chunk_idx in range(0, len(page_ranges), 5):  # 한 번에 5페이지씩 처리
                if collected_posts >= TOTAL_POSTS:
                    break
                
                # 현재 청크의 페이지 범위
                current_pages = page_ranges[page_chunk_idx:page_chunk_idx + 5]
                
                # 페이지별 게시물 목록 가져오기 (병렬 처리)
                all_posts_info = []
                with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
                    results = list(executor.map(extract_post_info, current_pages))
                
                for posts_in_page in results:
                    all_posts_info.extend(posts_in_page)
                
                # 샘플링 적용 (게시물 번호 기준으로 균등하게 샘플링)
                selected_posts = []
                for idx, post_info in enumerate(all_posts_info):
                    if (collected_posts + idx) % sampling_interval == 0:
                        selected_posts.append(post_info)
                        post_id_cache.add(post_info['post_id'])
                
                # 게시물 상세 정보 병렬 크롤링
                post_details_to_crawl = [(post['url'], post['post_id']) for post in selected_posts]
                
                # 최대 MAX_WORKERS 개의 스레드로 상세 페이지 병렬 크롤링
                crawled_details = []
                with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                    future_to_post = {executor.submit(crawl_post_detail, post_detail): post_detail 
                                     for post_detail in post_details_to_crawl}
                    
                    for future in concurrent.futures.as_completed(future_to_post):
                        result = future.result()
                        if result:
                            crawled_details.append(result)
                
                # 크롤링 결과 처리
                for detail in crawled_details:
                    # 상세 정보와 게시물 정보 매칭
                    matching_post = next((post for post in selected_posts if post['post_id'] == detail['post_id']), None)
                    
                    if matching_post:
                        new_row = {
                            '번호': matching_post['post_id'],
                            '날짜': matching_post['date'],
                            '제목': matching_post['title'],
                            '내용': detail['내용'],
                            '조회수': detail['조회수'],
                            '댓글갯수': detail['댓글갯수']
                        }
                        all_data.append(new_row)
                        collected_posts += 1
                        current_batch_size += 1
                        pbar.update(1)
                        
                        # 중간 저장
                        if current_batch_size >= SAVE_INTERVAL:
                            # 데이터프레임으로 변환하여 저장
                            df = pd.DataFrame(all_data)
                            save_to_csv(df, collected_posts)
                            current_batch_size = 0
                
                # 현재 크롤링 속도 계산 및 표시
                current_time = time.time()
                elapsed_time = current_time - start_time
                if elapsed_time > 0:
                    posts_collected = collected_posts - posts_at_start
                    posts_per_minute = (posts_collected / elapsed_time) * 60
                    print(f"현재 크롤링 속도: {posts_per_minute:.2f} 게시물/분 (총 {posts_collected}개 수집, {elapsed_time:.1f}초 소요)")
                
                # 페이지 처리 간 짧은 대기
                time.sleep(0.5)
        
    except KeyboardInterrupt:
        print("사용자에 의해 크롤링이 중단되었습니다.")
    finally:
        # 마지막 저장
        if all_data and current_batch_size > 0:
            df = pd.DataFrame(all_data)
            save_to_csv(df, collected_posts)
        
        # 최종 크롤링 속도 계산 및 표시
        total_time = time.time() - start_time
        posts_collected = collected_posts - posts_at_start
        if total_time > 0:
            posts_per_minute = (posts_collected / total_time) * 60
            print(f"평균 크롤링 속도: {posts_per_minute:.2f} 게시물/분")
            print(f"총 소요 시간: {total_time/60:.2f}분 ({total_time:.1f}초)")
        
        print(f"크롤링 완료: 총 {collected_posts}개 게시물 수집")

if __name__ == "__main__":
    main()

게시물 크롤링 진행:   8%|▊         | 1654/20000 [03:47<39:05,  7.82it/s]

현재 크롤링 속도: 462.27 게시물/분 (총 1753개 수집, 227.5초 소요)


게시물 크롤링 진행:   9%|▉         | 1754/20000 [04:00<39:22,  7.72it/s]

현재 크롤링 속도: 455.13 게시물/분 (총 1827개 수집, 240.9초 소요)


게시물 크롤링 진행:   9%|▉         | 1828/20000 [04:13<41:55,  7.22it/s]

...

현재 크롤링 속도: 365.73 게시물/분 (총 18042개 수집, 2959.9초 소요)


게시물 크롤링 진행:  90%|█████████ | 18043/20000 [49:27<04:49,  6.76it/s]

현재 크롤링 속도: 365.28 게시물/분 (총 18067개 수집, 2967.7초 소요)


게시물 크롤링 진행:  90%|█████████ | 18067/20000 [49:28<05:17,  6.09it/s]

dcinside_chartanalysis_data_18067.csv 파일에 18067개 게시물 저장 완료
평균 크롤링 속도: 365.21 게시물/분
총 소요 시간: 49.47분 (2968.2초)
크롤링 완료: 총 18067개 게시물 수집
